In [1]:
import numpy as np

In [24]:
articles = []
labels = []

with open('input/bbc_text_cls.csv') as f:
    running_art = ''
    for l in f.readlines()[1:]:
        if '",' not in l or '", ' in l:
            running_art += l.strip()
        else:
            last_line = l.split('",')
            assert len(last_line) == 2
            articles.append(running_art + last_line[0].strip())
            labels.append(last_line[1].strip())
            running_art = ''



In [25]:
punc = '''!()-[]{};:'"\,<>./?@#^&*_~'''

def remove_punc(s):
    ''' takes in a string, removes all "useless" punctuation, lowercases it '''
    no_punc = ''
    for char in s:
        if char not in punc:
            no_punc += char.lower()
    return no_punc

In [38]:
def articles_to_mm(articles):
    # mm structure will be mm[(word_idx-1, word_idx+1)] = [list of words that fit this pattern]
    # for start of line, will use an initial state distribution with structure isd
    isd = {}
    mm = {}

    for article in articles:
        words = remove_punc(article).split()
        assert len(words) > 0

        
        for idx, w in enumerate(words):

            # first word in line for generating the isd
            if idx == 0:
                next_w = words[idx+1]
                if next_w not in isd:
                    isd[next_w] = []
                isd[next_w].append(w)
            
            # generate mm for all words before the end
            elif idx < len(words) - 1:
                prev_w = words[idx-1]
                next_w = words[idx+1]

                if (prev_w, next_w) not in mm:
                    mm[(prev_w, next_w)] = []
                mm[(prev_w, next_w)].append(w)
            
            elif idx == len(words) -1:
                prev_w = words[idx-1]
                next_w = 'END'
                if (prev_w, next_w) not in mm:
                    mm[(prev_w, next_w)] = []
                mm[(prev_w, next_w)].append(w)
    
    return isd, mm

isd, mm = articles_to_mm(articles)

In [40]:
mm

{('ad', 'boost'): ['sales'],
 ('sales', 'time'): ['boost'],
 ('boost', 'warner'): ['time'],
 ('time', 'profitquarterly'): ['warner'],
 ('warner', 'profits'): ['profitquarterly'],
 ('profitquarterly', 'at'): ['profits'],
 ('profits', 'us'): ['at'],
 ('at', 'media'): ['us', 'the'],
 ('us', 'giant'): ['media',
  'telecoms',
  'foods',
  'oil',
  'mortgage',
  'agrochemical',
  'banking',
  'supermarket'],
 ('media', 'timewarner'): ['giant'],
 ('giant', 'jumped'): ['timewarner'],
 ('timewarner', '76'): ['jumped'],
 ('jumped', 'to'): ['76', '22'],
 ('76', '113bn'): ['to'],
 ('to', '£600m'): ['113bn'],
 ('113bn', 'for'): ['£600m'],
 ('£600m', 'the'): ['for'],
 ('for', 'three'): ['the',
  'the',
  'the',
  'the',
  'the',
  'the',
  'the',
  'just',
  'bbc',
  'almost',
  'almost',
  'nearly',
  'missing',
  'missing',
  'nearly',
  'the',
  'the',
  'the',
  'another',
  'the',
  'the',
  'the'],
 ('the', 'months'): ['three',
  'three',
  'three',
  'three',
  '12',
  'three',
  'three',
  '